In [81]:
from tensorflow.keras.models import load_model
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
loaded_model = load_model('lstm_model2.h5')

In [59]:
print(loaded_model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 300)          17834700  
                                                                 
 lstm (LSTM)                 (None, 100, 64)           93440     
                                                                 
 time_distributed (TimeDistr  (None, 100, 13)          845       
 ibuted)                                                         
                                                                 
Total params: 17,928,985
Trainable params: 94,285
Non-trainable params: 17,834,700
_________________________________________________________________
None


In [88]:
with open('data/data_normal.txt') as f:
    lines = f.readlines()

# tokenize the words
word = [[sent] for sent in lines]
# brown_sent = brown.tagged_sents(tagset='universal')
# word = [[word[0].lower() for word in tup] for tup in brown_sent]
word_tokenizer = Tokenizer()

word_tokenizer.fit_on_texts(word) 
word_seqs = word_tokenizer.texts_to_sequences(word[:100])  

max_len = 100 

# find the length of the data
w_size = len(word_tokenizer.word_index) + 1


word_padded = pad_sequences(word_seqs, max_len, padding='post', truncating='post') 

prediction = loaded_model.predict(word_padded)

4/4 [==============================] - 0s 17ms/step


In [75]:
from nltk.corpus import brown, treebank, conll2000

brown_sent = brown.tagged_sents(tagset='universal')
tree_sent = treebank.tagged_sents(tagset='universal')
conll_sent = conll2000.tagged_sents(tagset='universal')
all_sent = brown_sent + tree_sent + conll_sent
pos = [pos[1].lower() for tup in all_sent for pos in tup] # store the corresponding pos tag
pos_tokenizer = Tokenizer()
pos_tokenizer.fit_on_texts(pos)


In [89]:
paired = []
all_posttags = []
for p in prediction:
    pos_seq = [np.argmax(pred) for pred in p]
    pos_tags = [pos_tokenizer.sequences_to_texts([[i]])[0].upper() for i in pos_seq]
    pos_tags = list(filter(None, pos_tags))
    all_posttags.append(pos_tags)
#     print(pos_tags)
# for i in range(len(word)):
#     paired.append((word[i], all_posttags[i]))
# print(paired)

['ADJ', 'NOUN', 'NOUN', 'ADV', 'NOUN', 'DET', 'NOUN', 'ADP', 'ADV', 'DET', 'NOUN', 'NOUN', 'ADV', 'NOUN', 'NUM', 'ADP', 'CONJ', 'NOUN', 'CONJ', 'DET', 'ADV', 'ADV', 'ADV', 'VERB', 'ADP']
['ADJ', 'NOUN', 'NOUN', 'PRON', 'DET', 'NOUN', 'NOUN', 'CONJ', 'ADJ', 'ADV', 'VERB', 'NOUN', 'ADP', 'ADJ', 'VERB', 'NOUN', 'ADV', 'ADP', 'ADJ', 'NOUN', 'DET', 'ADV', 'ADJ', 'NOUN', 'DET', 'NOUN', 'ADP', 'ADJ', 'NOUN', 'ADP', 'VERB', 'CONJ', 'VERB', 'ADJ', 'NOUN', 'DET', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'VERB', 'DET']
['ADJ', 'VERB', 'NOUN', 'NOUN', 'VERB', 'ADJ', 'ADV', 'DET', 'NOUN', 'VERB', 'NOUN', 'NUM', 'ADV', 'NOUN', 'ADP', 'NOUN', 'PRON', 'ADP', 'VERB', 'ADJ', 'ADV', 'CONJ', 'DET', 'ADJ', 'NOUN', 'ADV', 'ADJ', 'VERB', 'ADV', 'DET', 'ADV', 'NOUN', 'NOUN', 'PRON', 'ADP']
['ADP', 'NOUN', 'DET', 'VERB', 'ADP', 'PRON', 'NOUN', 'VERB', 'ADV', 'CONJ', 'ADP', 'ADJ', 'NOUN', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADJ', 'NOUN', 'NOUN', 'DET', 'ADJ', 'NOUN', 'DET', 'ADJ', 'ADV', 'ADP', 'NOUN', 'DET', 'ADJ', 'ADV', 'AD

In [72]:
def accuracy(preds, labels):
    accuracies = []
    for i in range(len(preds)):
        actual = labels[i]
        predict = preds[i]
        acc = 0
        for j in range(len(predict)):
            try:
                if predict[j] == actual[j]:
                    acc += 1
            except:
                print('Line:', i)
                print('Predict:', len(predict))
                print('Actual:', len(actual))
        if len(preds[i]) > 0:
            acc = acc / len(preds[i])
        accuracies.append(acc)
    return accuracies

In [91]:
with open('data/labels_normal.txt') as f:
    pos_lines = f.readlines()
normal_pos = [labels.split() for labels in pos_lines]
# normal_pos = [[pos[1] for pos in tup] for tup in brown_sent]
print(accuracy(all_posttags, normal_pos))

Line: 7
Predict: 3
Actual: 2
Line: 11
Predict: 15
Actual: 14
Line: 18
Predict: 35
Actual: 34
Line: 24
Predict: 4
Actual: 3
Line: 25
Predict: 13
Actual: 12
Line: 27
Predict: 4
Actual: 3
Line: 34
Predict: 16
Actual: 15
Line: 38
Predict: 12
Actual: 11
Line: 50
Predict: 33
Actual: 32
Line: 51
Predict: 32
Actual: 31
Line: 53
Predict: 22
Actual: 21
Line: 54
Predict: 10
Actual: 9
Line: 59
Predict: 19
Actual: 18
Line: 61
Predict: 3
Actual: 2
Line: 65
Predict: 4
Actual: 3
Line: 72
Predict: 15
Actual: 13
Line: 72
Predict: 15
Actual: 13
Line: 78
Predict: 17
Actual: 16
Line: 81
Predict: 2
Actual: 1
Line: 91
Predict: 34
Actual: 33
Line: 96
Predict: 12
Actual: 11
[0.28, 0.19047619047619047, 0.2857142857142857, 0.027777777777777776, 0.16666666666666666, 0.18181818181818182, 0.3023255813953488, 0.3333333333333333, 0.2692307692307692, 0.12, 0.0, 0.26666666666666666, 0.10714285714285714, 0.21739130434782608, 0.288135593220339, 0.17391304347826086, 0.045454545454545456, 0.29411764705882354, 0.28571428571

In [96]:
print(normal_pos[0])
print(all_posttags[0])

['DET', 'NOUN', 'NOUN', 'ADJ', 'NOUN', 'VERB', 'NOUN', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADJ', 'NOUN', 'NOUN', 'VERB', '.', 'DET', 'NOUN', '.', 'ADP', 'DET', 'NOUN', 'VERB', 'NOUN', '.']
['ADJ', 'NOUN', 'NOUN', 'ADV', 'NOUN', 'DET', 'NOUN', 'ADP', 'ADV', 'DET', 'NOUN', 'NOUN', 'ADV', 'NOUN', 'NUM', 'ADP', 'CONJ', 'NOUN', 'CONJ', 'DET', 'ADV', 'ADV', 'ADV', 'VERB', 'ADP']


In [64]:
with open('data/labels_normal.txt') as f:
    pos_lines = f.readlines()
normal_pos = [labels.split() for labels in pos_lines]
print('Original:', accuracy(all_posttags, normal_pos))

Line: 7
Predict: 9
Actual: 8
Line: 16
Predict: 10
Actual: 6
Line: 16
Predict: 10
Actual: 6
Line: 16
Predict: 10
Actual: 6
Line: 16
Predict: 10
Actual: 6
Line: 17
Predict: 11
Actual: 8
Line: 17
Predict: 11
Actual: 8
Line: 17
Predict: 11
Actual: 8
Line: 21
Predict: 7
Actual: 6
Original: [0.0, 0.0, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.0, 0.0, 0.1111111111111111, 0.25, 0.5, 0.2, 0.3333333333333333, 0.0, 0.3333333333333333, 0.0, 0.25, 0.3, 0.09090909090909091, 0.2857142857142857, 0.0, 0.2, 0.2857142857142857]
